In [ ]:
# Notebook to duplicate ARGOG using cells 
## code is from main.py

In [23]:
# importing stuff
import openai
import chromadb
import pandas as pd
from utils import run_experiment, load_config
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, PromptTemplate, ServiceContext, Settings
# from llama_index.settings import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine, RetrieverQueryEngine
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.postprocessor import LLMRerank
from llama_index.core.retrievers import QueryFusionRetriever
from tonic_validate import ValidateScorer, ValidateApi
from tonic_validate.metrics import RetrievalPrecisionMetric, AnswerSimilarityMetric
import os

In [ ]:
### SETUP --------------------------------------------------------------------------------------------------------------
# Load the config file (.env vars)
load_config()

# Set the OpenAI API key for authentication.
openai.api_key = os.getenv("OPENAI_API_KEY")
tonic_validate_api_key = os.getenv("TONIC_VALIDATE_API_KEY")
tonic_validate_project_key = os.getenv("TONIC_VALIDATE_PROJECT_KEY")
tonic_validate_benchmark_key = os.getenv("TONIC_VALIDATE_BENCHMARK_KEY")
validate_api = ValidateApi(tonic_validate_api_key)


In [25]:
# Service context
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.0)
embed_model = OpenAIEmbedding(model="text-embedding-3-large")
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")
# service_context = ServiceContext.from_defaults(llm = llm, embed_model = embed_model)
# settings = Settings(llm=llm, embed_model=embed_model)
Settings.llm = llm 

chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [ ]:
# # existing_collections = chroma_client.list_collections()
# # print(existing_collections)
# # First, check if collection exists and create it if it doesn't
# try:
#     chroma_collection = chroma_client.get_collection("ai_arxiv_full")
# except:
#     # Create new collection
#     chroma_collection = chroma_client.create_collection(
#         name="ai_arxiv_full",
#         metadata={"description": "ArXiv AI papers collection"}
#     )

# # Now you can use the collection
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [33]:
from llama_index.core import Settings, StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import SentenceWindowNodeParser  # Updated import path

# Create the collection first
try:
    chroma_collection_sentence_window = chroma_client.get_collection("ai_arxiv_sentence_window")
except:
    chroma_collection_sentence_window = chroma_client.create_collection(
        name="ai_arxiv_sentence_window",
        metadata={"description": "ArXiv AI papers with sentence window processing"}
    )

# Set up sentence window processing
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text"
)

# Update settings with the node parser
Settings.node_parser = node_parser

# Set up vector store and index
vector_store_sentence_window = ChromaVectorStore(
    chroma_collection=chroma_collection_sentence_window
)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store_sentence_window
)

# Create the index
index_sentence_window = VectorStoreIndex.from_vector_store(
    vector_store=vector_store_sentence_window
)

In [29]:
# Traditional VDB
chroma_collection = chroma_client.get_collection("ai_arxiv_full")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [34]:
# Sentence window VDB
chroma_collection_sentence_window = chroma_client.get_collection("ai_arxiv_sentence_window")
vector_store_sentence_window = ChromaVectorStore(chroma_collection=chroma_collection_sentence_window)
index_sentence_window = VectorStoreIndex.from_vector_store(vector_store=vector_store_sentence_window)


In [35]:
# Document summary VDB
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

In [36]:
# storage_context = StorageContext.from_defaults(persist_dir="Obelix")
storage_context = StorageContext.from_defaults(persist_dir="ai_arxiv_doc_summary")
doc_summary_index = load_index_from_storage(llm=llm,
                                              storage_context=storage_context,
                                             embed_model = embed_model)

FileNotFoundError: [Errno 2] No such file or directory: 'e:/Dropbox/GithubRepo/StanfordContinuingStudies/TECH14LLMProduction/ARAGOG/ai_arxiv_doc_summary/docstore.json'